## Development notebook  FCN model 

Evaluate returned heatmap values from FCN, by passing data through MRCNN and then through FCN

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
%matplotlib inline
%load_ext autoreload
%autoreload 2
import os, sys, math, io, time, gc, argparse, platform, pprint
import numpy as np
import tensorflow as tf
import keras
import keras.backend as KB
sys.path.append('../')
import mrcnn.model_mrcnn  as mrcnn_modellib
import mrcnn.model_fcn    as fcn_modellib
import mrcnn.visualize    as visualize
import mrcnn.new_shapes   as shapes
import mrcnn.utils        as utils

from datetime           import datetime   
from mrcnn.utils        import command_line_parser, Paths
from mrcnn.config       import Config
from mrcnn.dataset      import Dataset 
from mrcnn.utils        import log   ##, stack_tensors, stack_tensors_3d, write_stdout
from mrcnn.datagen      import data_generator, load_image_gt, data_gen_simulate
from mrcnn.callbacks    import get_layer_output_1,get_layer_output_2
from mrcnn.coco         import CocoDataset, CocoConfig, CocoInferenceConfig, evaluate_coco, build_coco_results
from mrcnn.prep_notebook import mrcnn_newshape_train, prep_newshape_dataset,prep_coco_dataset

pp = pprint.PrettyPrinter(indent=2, width=100)
np.set_printoptions(linewidth=100,precision=4,threshold=1000, suppress = True)
start_time = datetime.now().strftime("%m-%d-%Y @ %H:%M:%S")
print()
print('--> Execution started at:', start_time)
print("    Tensorflow Version: {}   Keras Version : {} ".format(tf.__version__,keras.__version__))

####  Pass input parameters to argparse

# args = parser.parse_args("--epochs 100 --steps_in_epoch 128  --last_epoch 1264 --batch_size 8  --lr 0.5               --logs_dir train_fcn_adagrad --model /home/kbardool/models/train_mrcnn/shapes20180621T1554/mask_rcnn_shapes_1119.h5 --fcn_model init".split())
# input_parms = "--epochs 100 --steps_in_epoch 100  --last_epoch 1264 --batch_size 25 --lr 0.8 --val_steps 5 --logs_dir train_fcn_adagrad --model /home/kbardool/models/train_mrcnn/shapes20180621T1554/mask_rcnn_shapes_1119.h5 --fcn_model /home/kbardool/models/train_fcn_adagrad/shapes20180709T1732/fcn_shapes_1167.h5"
# input_parms +=" --model     /home/kbardool/models/train_mrcnn/shapes20180621T1554/mask_rcnn_shapes_1119.h5 "
##------------------------------------------------------------------------------------
## Parse command line arguments
##------------------------------------------------------------------------------------
parser = command_line_parser()
input_parms = "--epochs 2 "
input_parms += "--steps_in_epoch 2 " 
input_parms += "--val_steps 8 " 
input_parms += "--last_epoch 0 --batch_size 1 --lr 0.00001 "
# input_parms +="--mrcnn_logs_dir train_mrcnn_newshapes "
# input_parms +="--fcn_logs_dir   train_fcn8_newshapes "
input_parms +="--mrcnn_logs_dir train_mrcnn_coco "
input_parms +="--fcn_logs_dir   train_fcn8_coco "
input_parms +="--mrcnn_model    last "
input_parms +="--fcn_model      init "
input_parms +="--opt            adagrad "
input_parms +="--fcn_arch       fcn8 " 
input_parms +="--fcn_layers     all " 
input_parms +="--sysout        screen "
input_parms +="--new_log_folder    "
# input_parms +="--fcn_model /home/kbardool/models/train_fcn_adagrad/shapes20180709T1732/fcn_shapes_1167.h5"
print(input_parms)

args = parser.parse_args(input_parms.split())
# args = parser.parse_args()

##----------------------------------------------------------------------------------------------
## if debug is true set stdout destination to stringIO
##----------------------------------------------------------------------------------------------            
# debug = False
if args.sysout == 'FILE':
    sys.stdout = io.StringIO()

# print("    Dataset            : ", args.dataset)
# print("    Logs               : ", args.logs)
# print("    Limit              : ", args.limit)
print("    MRCNN Model        : ", args.mrcnn_model)
print("    FCN Model          : ", args.fcn_model)
print("    MRCNN Log Dir      : ", args.mrcnn_logs_dir)
print("    FCN Log Dir        : ", args.fcn_logs_dir)
print("    FCN Arch           : ", args.fcn_arch)
print("    FCN Log Dir        : ", args.fcn_layers)
print("    Last Epoch         : ", args.last_epoch)
print("    Epochs to run      : ", args.epochs)
print("    Steps in each epoch: ", args.steps_in_epoch)
print("    Validation steps   : ", args.val_steps)
print("    Batch Size         : ", args.batch_size)
print("    Optimizer          : ", args.opt)
print("    sysout             : ", args.sysout)
# print("    OS Platform        : ", syst)

##------------------------------------------------------------------------------------
## setup project directories
##   ROOT_DIR         : Root directory of the project 
##   MODEL_DIR        : Directory to save logs and trained model
##   COCO_MODEL_PATH  : Path to COCO trained weights
##---------------------------------------------------------------------------------
paths = Paths(fcn_training_folder = args.fcn_logs_dir, mrcnn_training_folder = args.mrcnn_logs_dir)
paths.display()

##------------------------------------------------------------------------------------
## Build configuration object 
##------------------------------------------------------------------------------------                          
mrcnn_config                    = CocoConfig()
# import mrcnn.new_shapes as new_shapes
# mrcnn_config = new_shapes.NewShapesConfig()

mrcnn_config.NAME               = 'mrcnn'              
mrcnn_config.TRAINING_PATH      = paths.MRCNN_TRAINING_PATH
mrcnn_config.COCO_DATASET_PATH  = paths.COCO_DATASET_PATH 
mrcnn_config.COCO_MODEL_PATH    = paths.COCO_MODEL_PATH   
mrcnn_config.RESNET_MODEL_PATH  = paths.RESNET_MODEL_PATH 
mrcnn_config.VGG16_MODEL_PATH   = paths.VGG16_MODEL_PATH  
mrcnn_config.COCO_CLASSES       = None 
mrcnn_config.DETECTION_PER_CLASS = 200
mrcnn_config.HEATMAP_SCALE_FACTOR = 4
mrcnn_config.BATCH_SIZE         = int(args.batch_size)                  # Batch size is 2 (# GPUs * images/GPU).
mrcnn_config.IMAGES_PER_GPU     = int(args.batch_size)                  # Must match BATCH_SIZE

mrcnn_config.STEPS_PER_EPOCH    = int(args.steps_in_epoch)
mrcnn_config.LEARNING_RATE      = float(args.lr)
mrcnn_config.EPOCHS_TO_RUN      = int(args.epochs)
mrcnn_config.FCN_INPUT_SHAPE    = mrcnn_config.IMAGE_SHAPE[0:2]
mrcnn_config.LAST_EPOCH_RAN     = int(args.last_epoch)

mrcnn_config.WEIGHT_DECAY       = 2.0e-4
mrcnn_config.VALIDATION_STEPS   = int(args.val_steps)
mrcnn_config.REDUCE_LR_FACTOR   = 0.5
mrcnn_config.REDUCE_LR_COOLDOWN = 30
mrcnn_config.REDUCE_LR_PATIENCE = 40
mrcnn_config.EARLY_STOP_PATIENCE= 80
mrcnn_config.EARLY_STOP_MIN_DELTA = 1.0e-4
mrcnn_config.MIN_LR             = 1.0e-10
mrcnn_config.OPTIMIZER          = args.opt.upper()
mrcnn_config.NEW_LOG_FOLDER       = True
mrcnn_config.SYSOUT               = args.sysout
mrcnn_config.display() 


###  Build Model

from mrcnn.prep_notebook import mrcnn_coco_train
mrcnn_model, mrcnn_config = mrcnn_coco_train(mode = 'trainfcn', mrcnn_config = mrcnn_config)

Using TensorFlow backend.



--> Execution started at: 11-08-2018 @ 11:47:41
    Tensorflow Version: 1.8.0   Keras Version : 2.1.6 
--epochs 2 --steps_in_epoch 2 --val_steps 8 --last_epoch 0 --batch_size 1 --lr 0.00001 --mrcnn_logs_dir train_mrcnn_coco --fcn_logs_dir   train_fcn8_coco --mrcnn_model    last --fcn_model      init --opt            adagrad --fcn_arch       fcn8 --fcn_layers     all --sysout        screen --new_log_folder    
    MRCNN Model        :  last
    FCN Model          :  init
    MRCNN Log Dir      :  train_mrcnn_coco
    FCN Log Dir        :  train_fcn8_coco
    FCN Arch           :  FCN8
    FCN Log Dir        :  ['all']
    Last Epoch         :  0
    Epochs to run      :  2
    Steps in each epoch:  2
    Validation steps   :  8
    Batch Size         :  1
    Optimizer          :  ADAGRAD
    sysout             :  SCREEN
>>> Initialize Paths
 Linx  Linux

Paths:
-------------------------
COCO_DATASET_PATH              /home/kbardool/MLDatasets/coco2014
COCO_MODEL_PATH                /h

     mrcnn_class_logits   output shape is :  (?, 200, 81)
     mrcnn_class_probs    output shape is :  (?, 200, 81)
     mrcnn_bbox_fc        output shape is :  (?, 200, 324)
     mrcnn_bbox_fc        reshaped output :  (?, 200, 324)
     mrcnn_bbox           output shape is :  (?, 200, 81, 4)
--------------------------------
>>>  CHM Layer  
--------------------------------
  > CHMLayer Call()  3
    mrcnn_class.shape    : (?, 200, 81) (None, 200, 81)
    mrcnn_bbox.shape     : (?, 200, 81, 4) (None, 200, 81, 4)
    output_rois.shape    : (1, ?, ?) (None, 200, 4)

  > build_predictions()
    num_rois               :  200
    norm_input_rois.shape  :  <class 'tensorflow.python.framework.ops.Tensor'> (None, 200, 4)
    scale.shape            :  <class 'tensorflow.python.framework.ops.Tensor'> (4,) (4,)
    dup_scale.shape        :  <class 'tensorflow.python.framework.ops.Tensor'> (1, 200, 4) (1, 200, 4)

    mrcnn_class shape      :  (None, 200, 81)
    mrcnn_bbox.shape       :  (None, 

In [2]:
# mrcnn_model.config.EPOCHS_TO_RUN = 1
mrcnn_model.config.display()  
mrcnn_model.layer_info()


Configuration Parameters:
-------------------------
BACKBONE_SHAPES                [[256 256]
 [128 128]
 [ 64  64]
 [ 32  32]
 [ 16  16]]
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COCO_CLASSES                   None
COCO_DATASET_PATH              /home/kbardool/MLDatasets/coco2014
COCO_MODEL_PATH                /home/kbardool/PretrainedModels/mask_rcnn_coco.h5
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
DETECTION_PER_CLASS            200
EARLY_STOP_MIN_DELTA           0.0001
EARLY_STOP_PATIENCE            80
EPOCHS_TO_RUN                  2
FCN_INPUT_SHAPE                [1024 1024]
GPU_COUNT                      1
HEATMAP_SCALE_FACTOR           4
IMAGES_PER_GPU                 1
IMAGE_MAX_DIM                  1024
IMAGE_MIN_DIM                  800
IMAGE_PADDING                  True
IMAGE_SHAPE                    [1024 1024    3]
LAS

##  Build FCN Model 

In [3]:
##------------------------------------------------------------------------------------
## Build configuration for FCN model
##------------------------------------------------------------------------------------
fcn_config = CocoConfig()
fcn_config.NAME                 = 'fcn'              
fcn_config.TRAINING_PATH        = paths.FCN_TRAINING_PATH
fcn_config.VGG16_MODEL_PATH     = paths.FCN_VGG16_MODEL_PATH
fcn_config.FCN_INPUT_SHAPE      = mrcnn_config.IMAGE_SHAPE[0:2] // mrcnn_config.HEATMAP_SCALE_FACTOR 

fcn_config.BATCH_SIZE           = int(args.batch_size)                  # Batch size is 2 (# GPUs * images/GPU).
fcn_config.IMAGES_PER_GPU       = int(args.batch_size)                  # Must match BATCH_SIZE
fcn_config.EPOCHS_TO_RUN        = int(args.epochs)
fcn_config.STEPS_PER_EPOCH      = int(args.steps_in_epoch)
fcn_config.LEARNING_RATE        = float(args.lr)
fcn_config.LAST_EPOCH_RAN       = int(args.last_epoch)
fcn_config.VALIDATION_STEPS     = int(args.val_steps)

fcn_config.WEIGHT_DECAY         = 2.0e-4     ## FCN Weight decays are 5.0e-4 or 2.0e-4
fcn_config.BATCH_MOMENTUM       = 0.9

fcn_config.REDUCE_LR_FACTOR     = 0.5
fcn_config.REDUCE_LR_COOLDOWN   = 15
fcn_config.REDUCE_LR_PATIENCE   = 50
fcn_config.REDUCE_LR_MIN_DELTA  = 1e-6

fcn_config.EARLY_STOP_PATIENCE  = 75
fcn_config.EARLY_STOP_MIN_DELTA = 1.0e-7

fcn_config.MIN_LR               = 1.0e-10
fcn_config.CHECKPOINT_PERIOD    = 1

fcn_config.NEW_LOG_FOLDER       = args.new_log_folder
fcn_config.OPTIMIZER            = args.opt.upper()
fcn_config.SYSOUT               = args.sysout
# fcn_config.display()

#### Build Model

In [4]:
##------------------------------------------------------------------------------------
## Build FCN Model in Training Mode
##------------------------------------------------------------------------------------
try :
    del fcn_model
    gc.collect()
except: 
    pass    
fcn_model = fcn_modellib.FCN(mode="training", arch = 'FCN8', config=fcn_config)


>>> Initialize ModelBase model 
   Mode      :  training
   Model dir :  /home/kbardool/models/train_fcn8_coco
>>> ModelBase initialiation complete
>>> Initialize FCN model, mode:  training architecture:  FCN8
>>> set_log_dir(): model_path:  None
    set_log_dir(): model_path has NOT been provided : None 
                  NewFolder: False  config.NEW_LOG_FOLDER: True 
    set_log_dir(): weight file template (self.checkpoint_path): /home/kbardool/models/train_fcn8_coco/fcn20181108T1147/fcn_{epoch:04d}.h5 
    set_log_dir(): weight file dir      (self.log_dir)        : /home/kbardool/models/train_fcn8_coco/fcn20181108T1147 
    set_log_dir(): Last completed epoch (self.epoch)          : 0 
    arch set to FCN8 - with No L2 Regularization
<function fcn8_graph at 0x7f221f2e18c8>
 Parse Image Meta Graph 
     meta :  <class 'tensorflow.python.framework.ops.Tensor'> (None, None)
 Parse Image Meta Graph 
     meta :  <class 'tensorflow.python.framework.ops.Tensor'> (None, None)
   active_cla

In [5]:
fcn_model.keras_model.summary()
fcn_model.layer_info()
# fcn_model.config.display()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_pr_hm_norm (InputLayer)   (None, 256, 256, 81) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 256, 256, 64) 46720       input_pr_hm_norm[0][0]           
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 256, 256, 64) 36928       block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, 128, 128, 64) 0           block1_conv2[0][0]               
__________________________________________________________________________________________________
block2_con

In [21]:
# del fcn_m
ff = fcn_model.keras_model
# pp.pprint(dir(ff))
# for i in ff.layers:
#     print(i.name)
ly = ff.layers[35]
dir(ly)
print(ly.input, ly.input_shape)
print(ly.output, ly.output_shape)
# test = [None] *len(fcn_m.targets)

print('ff.outputNames', ff.output_names)
print('ff.outputShapes', ff.output_shape)

print('ff.targets :', len(ff.targets))
for tensor in ff.targets:
      print('  ', tensor)
print('ff.inputs  :', len(ff.inputs))
for tensor in ff.inputs:
      print('   ', tensor)
print('fd.sample_weights :', ff.sample_weights)
print('ff.input          :', len(ff.input))
print('_feed_input_nmes: ', ff._feed_input_names)
print('_feed_output_nmes: ', ff._feed_output_names)

Tensor("input_image_meta_1:0", shape=(?, ?), dtype=float32) (None, None)
Tensor("active_class_ids/strided_slice:0", shape=(?, ?), dtype=float32) (None, None)
ff.outputNames ['fcn8_heatmap', 'fcn_MSE_loss', 'fcn_CE_loss']
ff.outputShapes [(None, 256, 256, 81), (1, 1), (1, 1)]
ff.targets : 3
   None
   None
   None
ff.inputs  : 5
    Tensor("input_image_meta_1:0", shape=(?, ?), dtype=float32)
    Tensor("input_pr_hm_norm:0", shape=(?, 256, 256, 81), dtype=float32)
    Tensor("input_pr_hm_scores:0", shape=(?, 81, 200, 11), dtype=float32)
    Tensor("input_gt_hm_norm:0", shape=(?, 256, 256, 81), dtype=float32)
    Tensor("input_gt_hm_scores:0", shape=(?, 81, 200, 11), dtype=float32)
fd.sample_weights : [None, None, None]
ff.input          : 5
_feed_input_nmes:  ['input_image_meta', 'input_pr_hm_norm', 'input_pr_hm_scores', 'input_gt_hm_norm', 'input_gt_hm_scores']
_feed_output_nmes:  []


## Display Images

In [ ]:
dataset_train, train_generator= prep_coco_dataset(['train','val35k'], mrcnn_config, generator = True)
dataset_val, val_generator   = prep_coco_dataset(['minival'], mrcnn_config, generator = True)

###  Display image with Ground Truth bounding boxes and masks

In [ ]:
## 62642 (persons),   68539 (trucks) 36466 (surfers)  75040 (boat and persons)
## 36466 surfers. 5498 basketbal players, 27711,30531
## 5498 lots of motorcylces & persons - 
## Persons: #26026, #7719, 111864, 58240,  
## 89243: Person, bicylce and traiffic lights
## 35347 - laptops, keyboards and cat
## items = [59199 , 102868]
## 101623 (cake and forks), 41423 (elephant & people)
from mrcnn.datagen  import data_gen_simulate
# [75040, 89243]
IMAGE_IDS = [11998]
# train_batch_x, train_batch_y = next(train_generator)
train_batch_x, train_batch_y = test_batch_x, test_batch_y = data_gen_simulate(dataset_train, mrcnn_config, IMAGE_IDS)
imgmeta_idx = mrcnn_model.keras_model.input_names.index('input_image_meta')
img_meta    = train_batch_x[imgmeta_idx]

for img_idx in range(mrcnn_config.BATCH_SIZE):
    image_id = img_meta[img_idx,0]
    image = dataset_train.load_image(image_id)
    mask, class_ids = dataset_train.load_mask(image_id)
    bbox = utils.extract_bboxes(mask)
    print('Image meta  : ', img_meta[img_idx,:10])
    print('Classes     : ', class_ids)
    print("image_id    : ", image_id, ' Reference: ', dataset_train.image_reference(image_id))
    print(' class_ids.shape[0]:', class_ids.shape[0], 'bbox.shape[0]:',bbox.shape[0])    
    
    class_names = [str(dataset_train.class_names[class_id]) for class_id in class_ids]
    print('Class Names : ', class_names)
    
    visualize.display_top_masks(image, mask, class_ids, dataset_train.class_names)   
    # Display image and instances
    visualize.display_instances_with_mask(image, bbox, mask, class_ids, dataset_train.class_names, figsize =(8,8))    

### other image displays

####  Display Training / Validation Training set information

In [ ]:
print("Train Dataset Image Count: {}".format(len(dataset_train.image_ids)))
print("Training Dataset Class Count: {}".format(dataset_train.num_classes))
for i, info in enumerate(dataset_train.class_info):
    print("{:3}. {:50}".format(i, info['name']))
    
    
print("Validation Dataset Image Count: {}".format(len(dataset_val.image_ids)))
print("Validation Dataset Class Count: {}".format(dataset_val.num_classes))
for i, info in enumerate(dataset_val.class_info):
    print("{:3}. {:50}".format(i, info['name']))    

#### Display top masks for a random group of images

In [ ]:
# Load and display random samples
image_ids = np.random.choice(dataset_train.image_ids, 7)
for image_id in image_ids:
    image = dataset_train.load_image(image_id)
    mask, class_ids = dataset_train.load_mask(image_id)
    visualize.display_top_masks(image, mask, class_ids, dataset_train.class_names)

#### Display a random image with instances and mask

In [ ]:
# Load random image and mask.
# image_id = np.random.choice(dataset_train.image_ids)


image    = dataset_train.load_image(image_id)
mask, class_ids = dataset_train.load_mask(image_id)
# Compute Bounding box
bbox = utils.extract_bboxes(mask)

# Display image and additional stats
print("image_id ", image_id, dataset_train.image_reference(image_id))
log("image", image)
log("mask", mask)
log("class_ids", class_ids)
log("bbox", bbox)
print(class_ids.shape[0], bbox.shape[0])
# Display image and instances
visualize.display_instances_with_mask(image, bbox, mask, class_ids, dataset_train.class_names)

##  Load Model Weights

In [6]:
# exclude=["mrcnn_class_logits"] # ,"mrcnn_bbox_fc"]   #, "mrcnn_bbox", "mrcnn_mask"])
mrcnn_model.load_model_weights(init_with = 'last', exclude = None)  

-----------------------------------------------
 Load Model with init parm: [ last ]
-----------------------------------------------
 ---> last
>>> find_last checkpoint in :  /home/kbardool/models/train_mrcnn_coco
    Dir starting with  mrcnn  : ['mrcnn20181105T1801', 'mrcnn20181107T1443', 'mrcnn20181107T1450', 'mrcnn20181105T1811', 'mrcnn20181105T1813', 'mrcnn20181011T1100']
    find_last():   dir_name: /home/kbardool/models/train_mrcnn_coco/mrcnn20181011T1100
    find_last(): checkpoint: /home/kbardool/models/train_mrcnn_coco/mrcnn20181011T1100/mrcnn_0103.h5
>>> load_weights() from : /home/kbardool/models/train_mrcnn_coco/mrcnn20181011T1100/mrcnn_0103.h5
layers type:  <class 'list'> length:  383
    Weights file loaded: /home/kbardool/models/train_mrcnn_coco/mrcnn20181011T1100/mrcnn_0103.h5 
MRCNN  MODEL Load weight file COMPLETE 


In [7]:
##------------------------------------------------------------------------------------
## Load FCN Model weights  
##------------------------------------------------------------------------------------
WEIGHTS_PATH = '/home/kbardool/models/train_fcn8_coco/fcn20181031T0000/fcn_0106.h5'
fcn_model.load_model_weights(init_with = WEIGHTS_PATH) # 'fcn_config.VGG16_MODEL_PATH')

-----------------------------------------------
 Load Model with init parm: [ /home/kbardool/models/train_fcn8_coco/fcn20181031T0000/fcn_0106.h5 ]
-----------------------------------------------
 ---> Explicit weight file
>>> load_weights() from : /home/kbardool/models/train_fcn8_coco/fcn20181031T0000/fcn_0106.h5
layers type:  <class 'list'> length:  38
    Weights file loaded: /home/kbardool/models/train_fcn8_coco/fcn20181031T0000/fcn_0106.h5 
FCN  MODEL Load weight file COMPLETE 


## FCN `train_in_batches()`

#### setup datasets

In [8]:
from mrcnn.prep_notebook import prep_coco_dataset
dataset_train = prep_coco_dataset(['val35k'], mrcnn_config)
dataset_val   = prep_coco_dataset(['minival'], mrcnn_config)

loading annotations into memory...
Done (t=4.72s)
creating index...
index created!
 image dir            :  /home/kbardool/MLDatasets/coco2014/val2014
 json_path_dir        :  /home/kbardool/MLDatasets/coco2014/annotations/instances_valminusminival2014.json
 number of images     :  35185
 image_ids[:10]       :  [262148, 393225, 131089, 262161, 262162, 393243, 262175, 131108, 393254, 42]
 image_ids[1000:1010] :  [134290, 527506, 527504, 134288, 527510, 265372, 265374, 265378, 527529, 3244]
loading annotations into memory...
Done (t=1.51s)
creating index...
index created!
 image dir            :  /home/kbardool/MLDatasets/coco2014/val2014
 json_path_dir        :  /home/kbardool/MLDatasets/coco2014/annotations/instances_minival2014.json
 number of images     :  4952
 image_ids[:10]       :  [532481, 458755, 245764, 385029, 311303, 393226, 532493, 475150, 458768, 8211]
 image_ids[1000:1010] :  [99053, 329219, 91654, 148999, 509451, 214539, 132622, 222735, 468501, 534041]


 #### Display parameters

In [9]:
from mrcnn.utils        import log
# fcn_config.EPOCHS_TO_RUN  = 4
# fcn_config.LEARNING_RATE  = 0.1
fcn_config.VALIDATION_STEPS = 4
print('--- MRCNN-------------------------------------------------------')
log("Epochs to run       {} ".format(mrcnn_model.config.EPOCHS_TO_RUN))
log("Steps per epochs    {} ".format(mrcnn_model.config.STEPS_PER_EPOCH))
log("Batch size          {} ".format(mrcnn_model.config.BATCH_SIZE))
log("Learning Rate       {} ".format(mrcnn_model.config.LEARNING_RATE))
log("Momentum            {} ".format(mrcnn_model.config.LEARNING_MOMENTUM))
log("Weight Decay:       {} ".format(mrcnn_model.config.WEIGHT_DECAY       ))
log("VALIDATION_STEPS    {} ".format(mrcnn_model.config.VALIDATION_STEPS   ))
# log("Checkpoint Path:    {} ".format(mrcnn_model.checkpoint_path))
# log("REDUCE_LR_FACTOR    {} ".format(mrcnn_model.config.REDUCE_LR_FACTOR   ))
# log("REDUCE_LR_COOLDOWN  {} ".format(mrcnn_model.config.REDUCE_LR_COOLDOWN ))
# log("REDUCE_LR_PATIENCE  {} ".format(mrcnn_model.config.REDUCE_LR_PATIENCE ))
# log("MIN_LR              {} ".format(mrcnn_model.config.MIN_LR             ))
# log("EARLY_STOP_PATIENCE {} ".format(mrcnn_model.config.EARLY_STOP_PATIENCE))     

print('--- FCN --------------------------------------------------------')
log("Epochs to run       {} ".format(fcn_model.config.EPOCHS_TO_RUN))
log("Steps per epochs    {} ".format(fcn_model.config.STEPS_PER_EPOCH))
log("Batch size          {} ".format(fcn_model.config.BATCH_SIZE))
log("Learning Rate       {} ".format(fcn_model.config.LEARNING_RATE))
log("Momentum            {} ".format(fcn_model.config.LEARNING_MOMENTUM))
log("Weight Decay:       {} ".format(fcn_model.config.WEIGHT_DECAY       ))
log("VALIDATION_STEPS    {} ".format(fcn_model.config.VALIDATION_STEPS   ))
log("Checkpoint Path:    {} ".format(fcn_model.checkpoint_path))
log("REDUCE_LR_FACTOR    {} ".format(fcn_model.config.REDUCE_LR_FACTOR   ))
log("REDUCE_LR_COOLDOWN  {} ".format(fcn_model.config.REDUCE_LR_COOLDOWN ))
log("REDUCE_LR_PATIENCE  {} ".format(fcn_model.config.REDUCE_LR_PATIENCE ))
log("MIN_LR              {} ".format(fcn_model.config.MIN_LR             ))
log("EARLY_STOP_PATIENCE {} ".format(fcn_model.config.EARLY_STOP_PATIENCE))    

--- MRCNN-------------------------------------------------------
Epochs to run       2 
Steps per epochs    2 
Batch size          1 
Learning Rate       1e-05 
Momentum            0.9 
Weight Decay:       0.0002 
VALIDATION_STEPS    8 
--- FCN --------------------------------------------------------
Epochs to run       2 
Steps per epochs    2 
Batch size          1 
Learning Rate       1e-05 
Momentum            0.9 
Weight Decay:       0.0002 
VALIDATION_STEPS    4 
Checkpoint Path:    /home/kbardool/models/train_fcn8_coco/fcn20181108T1147/fcn_{epoch:04d}.h5 
REDUCE_LR_FACTOR    0.5 
REDUCE_LR_COOLDOWN  15 
REDUCE_LR_PATIENCE  50 
MIN_LR              1e-10 
EARLY_STOP_PATIENCE 75 


#### Original Train_in_batches() call

In [ ]:
##----------------------------------------------------------------------------------------------
## Train the FCN only 
## Passing layers="heads" freezes all layers except the head
## layers. You can also pass a regular expression to select
## which layers to train by name pattern.
##----------------------------------------------------------------------------------------------            
# train_layers = ['all']
# loss_names   = ["fcn_MSE_loss"]
# fcn_config.LAST_EPOCH_RAN = 56
# fcn_model.epoch = fcn_config.LAST_EPOCH_RAN

# fcn_model.train_in_batches(
#             mrcnn_model,    
#             dataset_train,
#             dataset_val, 
#             layers = train_layers,
#             losses = loss_names
#             )


#### modified train_in_btches call - invokes compile2()

In [ ]:
##----------------------------------------------------------------------------------------------
## Train the FCN only 
## Passing layers="heads" freezes all layers except the head
## layers. You can also pass a regular expression to select
## which layers to train by name pattern.
##----------------------------------------------------------------------------------------------            
train_layers = ['all']
loss_names   = ["fcn_MSE_loss"]
fcn_config.LAST_EPOCH_RAN = 56
fcn_model.epoch = fcn_config.LAST_EPOCH_RAN

train_in_batches(fcn_model,
            mrcnn_model,    
            dataset_train,
            dataset_val, 
            layers = train_layers,
            losses = loss_names
            )


## Simulate Train in Batches - step by step

In [57]:
from mrcnn.standardize import *
# def train_in_batches(self,
#                 mrcnn_model,
#               train_dataset, 
#               val_dataset,  
#               layers            = None,
#               losses            = None,
train_layers      = ['all']
loss_names        = ["fcn_MSE_loss"]
fcn_config.LAST_EPOCH_RAN = 56
fcn_model.epoch   = fcn_config.LAST_EPOCH_RAN
self              = fcn_model
mrcnn_model       = mrcnn_model
train_dataset     = dataset_train
val_dataset       = dataset_val
layers            = train_layers
losses            = loss_names
learning_rate     = 0           
epochs            = 0
epochs_to_run     = 0 
batch_size        = 0 
steps_per_epoch   = 0
min_LR            = 0
debug             = False

'''
Train the model.
train_dataset, 
val_dataset:    Training and validation Dataset objects.

learning_rate:  The learning rate to train with

epochs:         Number of training epochs. Note that previous training epochs
                are considered to be done already, so this actually determines
                the epochs to train in total rather than in this particaular
                call.

layers:         Allows selecting wich layers to train. It can be:
                - A regular expression to match layer names to train
                - One of these predefined values:
                heads: The RPN, classifier and mask heads of the network
                all: All the layers
                3+: Train Resnet stage 3 and up
                4+: Train Resnet stage 4 and up
                5+: Train Resnet stage 5 and up
'''
assert self.mode == "training", "Create model in training mode."

if batch_size == 0 :
    batch_size = self.config.BATCH_SIZE

if epochs_to_run ==  0 :
    epochs_to_run = self.config.EPOCHS_TO_RUN

if steps_per_epoch == 0:
    steps_per_epoch = self.config.STEPS_PER_EPOCH

if min_LR == 0 :
    min_LR = self.config.MIN_LR

if learning_rate == 0:
    learning_rate = self.config.LEARNING_RATE

epochs = self.epoch + epochs_to_run

# use Pre-defined layer regular expressions
# if layers in self.layer_regex.keys():
    # layers = self.layer_regex[layers]
print(layers)
# train_regex_list = []
# for x in layers:
    # print( ' layers ias : ',x)
    # train_regex_list.append(x)
train_regex_list = [self.layer_regex[x] for x in layers]
print(train_regex_list)
layers = '|'.join(train_regex_list)        
print('layers regex :', layers)


##--------------------------------------------------------------------------------
## Data generators
##--------------------------------------------------------------------------------
train_generator = data_generator(train_dataset, mrcnn_model.config, shuffle=True,
                                 batch_size=batch_size)
val_generator   = data_generator(val_dataset, mrcnn_model.config, shuffle=True,
                                 batch_size=batch_size,
                                 augment=False)

##--------------------------------------------------------------------------------
## Set trainable layers and compile
##--------------------------------------------------------------------------------
self.set_trainable(layers)            

##----------------------------------------------------------------------------------------------
## Setup optimizaion method 
##----------------------------------------------------------------------------------------------            
optimizer = self.set_optimizer()

# self.compile(learning_rate, self.config.LEARNING_MOMENTUM, losses)        
self.compile(losses, optimizer)

##--------------------------------------------------------------------------------
## get metrics from keras_model.metrics_names and setup callback metrics 
##--------------------------------------------------------------------------------
out_labels = self.get_deduped_metrics_names()
callback_metrics = out_labels + ['val_' + n for n in out_labels]

print()
print(' Post-compile out_labels from get_deduped_metrics_names() : ')
print(' ---------------------------------------------------------- ')
for i in out_labels:
    print('     -',i)
print()
print(' Post-compile Callback metrics monitored by progbar :')
print(' ----------------------------------------------------')
for i in callback_metrics:
    print('     -',i)

print()
print(' Post-compile Keras metric_names :') 
print(' ---------------------------------') 
for idx, i in enumerate(self.keras_model.metrics_names):
    print('     ',idx, '  ', i)

print()
print(' Post-compile Keras stateful_metric_names :') 
print(' ------------------------------------------') 
for idx, i in enumerate(self.keras_model.stateful_metric_names):
    print('     ',idx, '  ', i)

## Setup for stateful_metric_indices Validation process 
##--------------------------------------------------------------------------------
stateful_metric_indices = []
if hasattr(self, 'metrics'):
    for m in self.stateful_metric_functions:
        m.reset_states()
    stateful_metric_indices = [
        i for i, name in enumerate(self.metrics_names)
        if str(name) in self.stateful_metric_names]
else:
    stateful_metric_indices = []

##--------------------------------------------------------------------------------
## Callbacks
##--------------------------------------------------------------------------------
# call back for model checkpoint was originally (?) loss. chanegd to val_loss (which is default) 2-5-18
# copied from \keras\engine\training.py
# def _get_deduped_metrics_names(self):


callbacks_list = [
      keras.callbacks.ProgbarLogger(count_mode='steps',
                                    stateful_metrics=self.keras_model.stateful_metric_names)

    , keras.callbacks.BaseLogger(stateful_metrics=self.keras_model.stateful_metric_names)

    , keras.callbacks.TensorBoard(log_dir=self.log_dir,
                                  histogram_freq=1,
                                  write_graph=True,
                                  write_images=False, 
                                  write_grads=True,
                                  batch_size=self.config.BATCH_SIZE)
                                  # write_graph=True,

                                  # write_images=True,
                                  # embeddings_freq=0,
                                  # embeddings_layer_names=None,
                                  # embeddings_metadata=None)

    , keras.callbacks.ModelCheckpoint(self.checkpoint_path, 
                                      mode    = 'auto', 
                                      period  = self.config.CHECKPOINT_PERIOD, 
                                      monitor = 'val_loss', 
                                      verbose = 1, 
                                      save_best_only = True, 
                                      save_weights_only=True)

    , keras.callbacks.ReduceLROnPlateau(monitor='val_loss', 
                                        mode     = 'auto', 
                                        factor   = self.config.REDUCE_LR_FACTOR,   
                                        cooldown = self.config.REDUCE_LR_COOLDOWN,
                                        patience = self.config.REDUCE_LR_PATIENCE,
                                        min_delta= self.config.REDUCE_LR_MIN_DELTA,
                                        min_lr   = self.config.MIN_LR, 
                                        verbose  = 1)                                            

    , keras.callbacks.EarlyStopping(monitor='val_loss', 
                                        mode      = 'auto', 
                                        min_delta = self.config.EARLY_STOP_MIN_DELTA, 

                                        patience  = self.config.EARLY_STOP_PATIENCE, 
                                        verbose   = 1)                                            
    , keras.callbacks.History() 
]


callbacks =  keras.callbacks.CallbackList(callbacks = callbacks_list)
callbacks.set_model(self.keras_model)
callbacks.set_params({
    'batch_size': batch_size,
    'epochs': epochs,
    'steps': steps_per_epoch,
    'verbose': 1 ,
    'do_validation': True,
    'metrics': callback_metrics
})

    # 'samples': num_train_samples,
    # 'verbose': verbose,
    # 'do_validation': do_validation,
    # 'metrics': callback_metrics or [],

log(" ")
log("Training Start Parameters:")
log("--------------------------")
log("Starting at epoch     {} of {} epochs.".format(self.epoch, epochs))
log("Steps per epochs      {} ".format(steps_per_epoch))
log("Last epoch completed  {} ".format(self.epoch))
log("Batch size            {} ".format(batch_size))
log("Learning Rate         {} ".format(self.config.LEARNING_RATE))
log("Momentum              {} ".format(self.config.LEARNING_MOMENTUM))
log("Weight Decay:         {} ".format(self.config.WEIGHT_DECAY       ))
log("VALIDATION_STEPS      {} ".format(self.config.VALIDATION_STEPS   ))
log("REDUCE_LR_FACTOR      {} ".format(self.config.REDUCE_LR_FACTOR   ))
log("REDUCE_LR_COOLDOWN    {} ".format(self.config.REDUCE_LR_COOLDOWN ))
log("REDUCE_LR_PATIENCE    {} ".format(self.config.REDUCE_LR_PATIENCE ))
log("MIN_LR                {} ".format(self.config.MIN_LR             ))
log("EARLY_STOP_PATIENCE   {} ".format(self.config.EARLY_STOP_PATIENCE))        
log("Checkpoint Path:      {} ".format(self.checkpoint_path))


##----------------------------------------------------------------------------------------------
## If in debug mode write stdout intercepted IO to output file  
##----------------------------------------------------------------------------------------------            
if self.config.SYSOUT == 'FILE':
    utils.write_sysout(self.log_dir)

##--------------------------------------------------------------------------------
## Start main training loop
##--------------------------------------------------------------------------------
early_stopping  = False
val_steps = self.config.VALIDATION_STEPS
epoch_idx = self.epoch
callbacks.on_train_begin()

['all']
['.*']
layers regex : .*

Selecting layers to train
-------------------------
Layer    Layer Name               Layer Type
   0  input_pr_hm_norm       (InputLayer          )   ............................no weights to train ]
   1  block1_conv1           (Conv2D              )   TRAIN 
   2  block1_conv2           (Conv2D              )   TRAIN 
   3  block1_pool            (MaxPooling2D        )   ............................no weights to train ]
   4  block2_conv1           (Conv2D              )   TRAIN 
   5  block2_conv2           (Conv2D              )   TRAIN 
   6  block2_pool            (MaxPooling2D        )   ............................no weights to train ]
   7  block3_conv1           (Conv2D              )   TRAIN 
   8  block3_conv2           (Conv2D              )   TRAIN 
   9  block3_conv3           (Conv2D              )   TRAIN 
  10  block3_pool            (MaxPooling2D        )   ............................no weights to train ]
  11  block4_conv1        

 grads:  <class 'list'>
   mapped_weight_name :  block4_conv1/bias_0
   write grads
 grads:  <class 'list'>
   write output layer  block4_conv1  type(output):  <class 'tensorflow.python.framework.ops.Tensor'>
layer:  block4_conv2
   mapped_weight_name :  block4_conv2/kernel_0
   write grads
 grads:  <class 'list'>
   mapped_weight_name :  block4_conv2/bias_0
   write grads
 grads:  <class 'list'>
   write output layer  block4_conv2  type(output):  <class 'tensorflow.python.framework.ops.Tensor'>
layer:  block4_conv3
   mapped_weight_name :  block4_conv3/kernel_0
   write grads
 grads:  <class 'list'>
   mapped_weight_name :  block4_conv3/bias_0
   write grads
 grads:  <class 'list'>
   write output layer  block4_conv3  type(output):  <class 'tensorflow.python.framework.ops.Tensor'>
layer:  block4_pool
   write output layer  block4_pool  type(output):  <class 'tensorflow.python.framework.ops.Tensor'>
layer:  block5_conv1
   mapped_weight_name :  block5_conv1/kernel_0
   write grads
 gra

In [58]:
if epoch_idx >= epochs:
    print('Final epoch {} has already completed - Training will not proceed'.format(epochs))
else:

    while epoch_idx < epochs :
        callbacks.on_epoch_begin(epoch_idx)
        epoch_logs = {}

        ##------------------------------------------------------------------------
        ## TRAINING Phase - emulating fit_generator()
        ##------------------------------------------------------------------------
        for steps_index in range(steps_per_epoch):

            # print(' self.epoch {}   epochs {}  step {} '.format(self.epoch, epochs, steps_index))
            batch_logs = {}
            batch_logs['batch'] = steps_index
            batch_logs['size']  = batch_size    

            callbacks.on_batch_begin(steps_index, batch_logs)

            train_batch_x, train_batch_y = next(train_generator)


            # print('len of train batch x' ,len(train_batch_x))
            # for idx, i in  enumerate(train_batch_x):
                # print(idx, 'type: ', type(i), 'shape: ', i.shape)
            # print('len of train batch y' ,len(train_batch_y))
            # for idx, i in  enumerate(train_batch_y):
                # print(idx, 'type: ', type(i), 'shape: ', i.shape)
            # print(type(output_rois))
            # for i in model_output:
                # print( i.shape)       

            ## Run prediction on MRCNN  
            try:
                results = mrcnn_model.keras_model.predict(train_batch_x)
                fcn_x = [train_batch_x[1]]
                fcn_x.extend(results[:4])

            except Exception as e :
                print('failure on mrcnn predict - epoch {} , image ids: {} '.format(epoch_idx, train_batch_x[1][:,0]))
                print('Exception information:')
                print(str(e))

            # print('size of results : ', len(results))
            # for idx, i in  enumerate(x):
                # print(idx, 'type: ', type(i), 'shape: ', i.shape)

            ## Train on FCN
            try:
                outs = self.keras_model.train_on_batch(fcn_x , train_batch_y)                                            
            except Exception as e :
                print('failure on fcn train - epoch {} , image ids: {} '.format(epoch_idx, train_batch_x[1][:,0]))
                print('Exception information:')
                print(str(e))                

            # print('size of outputs from train_on_batch : ', len(outs), outs)
            # for idx, i in  enumerate(outs):
                # print(idx, 'type: ', type(i), 'shape: ', i.shape)

            if not isinstance(outs, list):
                outs = [outs]

            for l, o in zip(out_labels, outs):
                # print(' out label: ', l, ' out value: ', o,' shape: ', o.shape)
                batch_logs[l] = o

            callbacks.on_batch_end(steps_index, batch_logs)
            epoch_idx +=1

Epoch 57/58
1/2 [==============>...............] - ETA: 7s - loss: 0.0132 - fcn_MSE_loss: 0.0132

In [59]:
##------------------------------------------------------------------------
## VALIDATION Phase - emulating evaluate_generator()
##------------------------------------------------------------------------
# print(' Start validation ')
# print(' ---------------- ')
# print(' Stateful metric indices:' )
# pp.pprint(stateful_metric_indices)


val_steps_done      = 0
val_outs_per_batch  = []
val_batch_sizes     = []

# setup validation progress bar if we wish
# progbar = Progbar(target=val_steps)

while val_steps_done < val_steps:
    # print(' ** Validation step: ', val_steps_done)

    mrcnn_val_x, mrcnn_val_y = next(val_generator)

    # print('len of train batch x' ,len(val_x))
    # for idx, i in  enumerate(val_x):
        # print(idx, 'type: ', type(i), 'shape: ', i.shape)

    ## Run prediction on MRCNN  
    try:
        val_results = mrcnn_model.keras_model.predict(mrcnn_val_x)
        fcn_val_x = [mrcnn_val_x[1]]
        fcn_val_x.extend(val_results[:4])   ## image_meta, pr_hm, pr_hm_scores, gt_hm, gt_hm_scores
    except Exception as e :
        print('failure on mrcnn predict (validation)- epoch {} , image ids: {} '.format(epoch_idx, mrcnn_val_x[1][:,0]))
        print('Exception information:')
        print(str(e))                

    # print('    mrcnn_model.predict() size of results : ', len(val_results))
    # for idx, i in  enumerate(xval_results):
        # print('    ',idx, 'type: ', type(i), 'shape: ', i.shape)

    ## Train on FCN
    try:
        outs2 = self.keras_model.test_on_batch( fcn_val_x , mrcnn_val_y)
        # print('\n valstep {} outs2 len:{} '.format(val_steps_done, len(outs2)))
        val_outs_per_batch.append(outs2)
        print('fcn_model.test_on_batch() size of results : ', len(outs2))
        for idx, i in  enumerate(outs2):
            print(idx, 'type: ', type(i), 'shape: ', i.shape, 'val: ', i)
    except Exception as e :
        print('failure on fcn train (validation)- epoch {} , image ids: {} '.format(epoch_idx, mrcnn_val_x[1][:,0]))                    
        print('Exception information:')
        print(str(e))                



    if isinstance(fcn_val_x, list):
        batch_size = fcn_val_x[0].shape[0]
    elif isinstance(fcn_val_x, dict):
        batch_size = list(fcn_val_x.values())[0].shape[0]
    else:
        batch_size = fcn_val_x.shape[0]

    if batch_size == 0:
        raise ValueError('Received an empty batch. '
                         'Batches should at least contain one item.')
    # else:
        # print('batch size:', batch_size)

    val_steps_done += 1
    val_batch_sizes.append(batch_size)
    # print validation progress bar if we wish
    # progbar.update(val_steps_done)

## calculate val_averages after all validations steps complete, which is passed 
## back to fit_generator() as val_outs 

fcn_model.test_on_batch() size of results :  2
0 type:  <class 'numpy.ndarray'> shape:  (1, 1) val:  [[0.0039]]
1 type:  <class 'numpy.ndarray'> shape:  (1, 1) val:  [[0.0039]]


/home/kbardool/anaconda3/envs/TFG/lib/python3.5/site-packages/scipy/ndimage/interpolation.py:616: UserWarning: From scipy 0.13.0, the output shape of zoom() is calculated with round() instead of int() - for these inputs the size of the returned array has changed.
  "the returned array has changed.", UserWarning)


fcn_model.test_on_batch() size of results :  2
0 type:  <class 'numpy.ndarray'> shape:  (1, 1) val:  [[0.0038]]
1 type:  <class 'numpy.ndarray'> shape:  (1, 1) val:  [[0.0038]]
fcn_model.test_on_batch() size of results :  2
0 type:  <class 'numpy.ndarray'> shape:  (1, 1) val:  [[0.0036]]
1 type:  <class 'numpy.ndarray'> shape:  (1, 1) val:  [[0.0036]]
fcn_model.test_on_batch() size of results :  2
0 type:  <class 'numpy.ndarray'> shape:  (1, 1) val:  [[0.0038]]
1 type:  <class 'numpy.ndarray'> shape:  (1, 1) val:  [[0.0038]]


In [60]:
print('    val_batch_sizes            :', type(val_batch_sizes),' len :', len(val_batch_sizes), val_batch_sizes)
print('    val_batch_sizes-shape      :', np.asarray(val_batch_sizes).shape)

print('    val_outs_per_batch:        :', type(val_outs_per_batch),' len :', len(val_outs_per_batch))
print('    val_outs_per_batch - shape :', np.asarray(val_outs_per_batch).shape)
for i,j in enumerate(val_outs_per_batch):
    print('        batch: ', i, '  ', j)

val_averages = []
for i in range(len(outs2)):
    if i not in stateful_metric_indices:
        tt = [out[i] for out in val_outs_per_batch]
        # print(' tt type: ',type(tt), tt)
        # print('val_batch_sizes.shape' , type(val_batch_sizes), len(val_batch_sizes))
        val_averages.append(
                np.average([out[i] for out in val_outs_per_batch], axis = 0, weights=val_batch_sizes)
                           )
    else:
        val_averages.append(float(val_outs_per_batch[-1][i]))
if len(val_averages) == 1:
    val_averages = val_averages[0]
print()
print('val_averages :', val_averages)
print()

    val_batch_sizes            : <class 'list'>  len : 4 [1, 1, 1, 1]
    val_batch_sizes-shape      : (4,)
    val_outs_per_batch:        : <class 'list'>  len : 4
    val_outs_per_batch - shape : (4, 2, 1, 1)
        batch:  0    [array([[0.0039]], dtype=float32), array([[0.0039]], dtype=float32)]
        batch:  1    [array([[0.0038]], dtype=float32), array([[0.0038]], dtype=float32)]
        batch:  2    [array([[0.0036]], dtype=float32), array([[0.0036]], dtype=float32)]
        batch:  3    [array([[0.0038]], dtype=float32), array([[0.0038]], dtype=float32)]

val_averages : [array([[0.0038]]), array([[0.0038]])]



In [80]:
#-- (unsuccessful) attempt to add histogram info to tensoflow summary  ------------------------
print(' Tensordlow histogram attempt')
print('-----------------------------')
print('model targets:', self.keras_model.targets)
print(' Total loss  :', self.keras_model.total_loss)
print(' Metric Tensors:', self.keras_model.metrics_tensors)
fcn_val_y = self.keras_model.targets
fcn_val_sample_weights = self.keras_model.sample_weights


print(' len(fcn_val_x)  : ',len(fcn_val_x))
print(' len(fcn_val_y)  : ',len(fcn_val_y))
print(' len(mrcnn_val_y): ',len(mrcnn_val_y))

# fcn_val_x, fcn_val_y, fcn_val_sample_weights = my_standardize_user_data(self.keras_model, fcn_val_x, fcn_val_y, val_sample_weight)
fcn_val_data = fcn_val_x + fcn_val_y  + fcn_val_sample_weights

print(' len(fcn_val_x)             : ',len(fcn_val_x))
print(' len(fcn_val_y)             : ',len(fcn_val_y))
print(' len(fcn_val_sample_weights): ',len(fcn_val_sample_weights))
print(' len(fcn_val_data)          : ',len(fcn_val_data))
if self.keras_model.uses_learning_phase and not isinstance(KB.learning_phase(), int):
    print(' add ')
    fcn_val_data += [0.]
for cbk in callbacks:
    cbk.validation_data = fcn_val_data

#-------------------------------------------------------------------------------

 Tensordlow histogram attempt
-----------------------------
model targets: [None, None, None]
 Total loss  : Tensor("loss_1/add:0", shape=(1, 1), dtype=float32)
 Metric Tensors: [<tf.Tensor 'Mean_3:0' shape=(1, 1) dtype=float32>]
 len(fcn_val_x)  :  5
 len(fcn_val_y)  :  3
 len(mrcnn_val_y):  0
 len(fcn_val_x)             :  5
 len(fcn_val_y)             :  3
 len(fcn_val_sample_weights):  3
 len(fcn_val_data)          :  11
 add 


In [81]:
# pp.pprint(val_averages)
# print(out_labels)
# epoch_logs = {}
# print(epoch_logs)
print(self.keras_model.inputs)
print(self.keras_model.targets)
print(self.keras_model.sample_weights)
print(self.keras_model.uses_learning_phase)

[<tf.Tensor 'input_image_meta_1:0' shape=(?, ?) dtype=float32>, <tf.Tensor 'input_pr_hm_norm:0' shape=(?, 256, 256, 81) dtype=float32>, <tf.Tensor 'input_pr_hm_scores:0' shape=(?, 81, 200, 11) dtype=float32>, <tf.Tensor 'input_gt_hm_norm:0' shape=(?, 256, 256, 81) dtype=float32>, <tf.Tensor 'input_gt_hm_scores:0' shape=(?, 81, 200, 11) dtype=float32>]
[None, None, None]
[None, None, None]
True


In [82]:
##------------------------------------------------------------------------
## END OF EPOCH Phase 
##------------------------------------------------------------------------
## end of evaluate_generator() emulation
## val_averages returned back to fit_generator() as val_outs
## calculate val_outs after all validations steps complete
##------------------------------------------------------------------------
if not isinstance(val_averages, list):
    val_averages = [val_averages]
# Same labels assumed.
for l, o in zip(out_labels, val_averages):
    epoch_logs['val_' + l] = o

#----commented 31-10-18 replaced with above lines -------------------------------------------
# if not isinstance(outs2, list):
    # val_outs =  np.average(np.asarray(val_all_outs), weights=val_batch_sizes)
# else:
    # averages = []
    # for i in range(len(outs2)):
        # averages.append(np.average([out[i] for out in val_all_outs], axis = 0, weights=val_batch_sizes))
    # val_outs = averages
# if not isinstance(val_outs, list):
    # val_outs = [val_outs]

# # Same labels assumed.
# for l, o in zip(out_labels, val_outs):
    # # print(' Validations : out label: val_', l, ' out value: ', o)
    # epoch_logs['val_' + l] = o
#-------------------------------------------------------------------------------------

# write_log(callback, val_names, logs, batch_no//10)
# print('\n    validation logs output: ', val_outs)


epoch_logs.update({'lr': KB.eval(self.keras_model.optimizer.lr)})    
callbacks.on_epoch_end(epoch_idx, epoch_logs)
epoch_idx += 1
                

2/2 [==============================] - 717s 359s/step - loss: 0.0157 - fcn_MSE_loss: 0.0157 - val_loss: 0.0038 - val_fcn_MSE_loss: 0.0038

len(self.model.inputs)  : 5 len(self.model.targets) : 3 len(self.model.sample_weights) : 3
 len(val_data):  12  len(tensors):  12


TypeError: 'NoneType' object is not subscriptable

In [ ]:
                for callback in callbacks:
                    # print(callback)
                    # pp.pprint(dir(callback.model))
                    if hasattr(callback.model, 'stop_training') and (callback.model.stop_training ==True):
                        print(' +++++++++++ ON EPOCH END CALLBACKS TRIGGERED STOP_TRAINING +++++++++++++')
                        print(callback.model, ' triggered stop_training +++++++++++++')
                        early_stopping = True
                        
                if early_stopping:
                    print('{}  Early Stopping triggered on epoch {} of {} epochs'.format(callback, epoch_idx, epochs))
                    break    
                
            ##-------------------------------
            ## end of training operations
            ##--------------------------------
            # if epoch_idx != self.epoch:
            # chkpoint.on_epoch_end(epoch_idx -1, batch_logs)
            callbacks.on_train_end()
            self.epoch = max(epoch_idx - 1, epochs)
            print('Final : self.epoch {}   epochs {}'.format(self.epoch, epochs))
            
        ##--------------------------------------------------------------------------------
        ## End main training loop
        ##--------------------------------------------------------------------------------
        return 
        

## Simulate Train in Batches

In [ ]:


def train_in_batches(self,
                mrcnn_model,
              train_dataset, 
              val_dataset,  
              layers            = None,
              losses            = None,
              learning_rate     = 0,              
              epochs            = 0,
              epochs_to_run     = 0, 
              batch_size        = 0, 
              steps_per_epoch   = 0,
              min_LR            = 0,
              debug             = False):
              
        '''
        Train the model.
        train_dataset, 
        val_dataset:    Training and validation Dataset objects.
        
        learning_rate:  The learning rate to train with
        
        epochs:         Number of training epochs. Note that previous training epochs
                        are considered to be done already, so this actually determines
                        the epochs to train in total rather than in this particaular
                        call.
                        
        layers:         Allows selecting wich layers to train. It can be:
                        - A regular expression to match layer names to train
                        - One of these predefined values:
                        heads: The RPN, classifier and mask heads of the network
                        all: All the layers
                        3+: Train Resnet stage 3 and up
                        4+: Train Resnet stage 4 and up
                        5+: Train Resnet stage 5 and up
        '''
        assert self.mode == "training", "Create model in training mode."
        
        if batch_size == 0 :
            batch_size = self.config.BATCH_SIZE
        
        if epochs_to_run ==  0 :
            epochs_to_run = self.config.EPOCHS_TO_RUN
        
        if steps_per_epoch == 0:
            steps_per_epoch = self.config.STEPS_PER_EPOCH

        if min_LR == 0 :
            min_LR = self.config.MIN_LR
        
        if learning_rate == 0:
            learning_rate = self.config.LEARNING_RATE
            
        epochs = self.epoch + epochs_to_run
            
        # use Pre-defined layer regular expressions
        # if layers in self.layer_regex.keys():
            # layers = self.layer_regex[layers]
        print(layers)
        # train_regex_list = []
        # for x in layers:
            # print( ' layers ias : ',x)
            # train_regex_list.append(x)
        train_regex_list = [self.layer_regex[x] for x in layers]
        print(train_regex_list)
        layers = '|'.join(train_regex_list)        
        print('layers regex :', layers)
        
        
        ##--------------------------------------------------------------------------------
        ## Data generators
        ##--------------------------------------------------------------------------------
        train_generator = data_generator(train_dataset, mrcnn_model.config, shuffle=True,
                                         batch_size=batch_size)
        val_generator   = data_generator(val_dataset, mrcnn_model.config, shuffle=True,
                                         batch_size=batch_size,
                                         augment=False)

        ##--------------------------------------------------------------------------------
        ## Set trainable layers and compile
        ##--------------------------------------------------------------------------------
        self.set_trainable(layers)            

        ##----------------------------------------------------------------------------------------------
        ## Setup optimizaion method 
        ##----------------------------------------------------------------------------------------------            
        optimizer = self.set_optimizer()

        # self.compile(learning_rate, self.config.LEARNING_MOMENTUM, losses)        
        self.compile(losses, optimizer)

        ##--------------------------------------------------------------------------------
        ## get metrics from keras_model.metrics_names and setup callback metrics 
        ##--------------------------------------------------------------------------------
        out_labels = self.get_deduped_metrics_names()
        callback_metrics = out_labels + ['val_' + n for n in out_labels]
        
        print()
        print(' Post-compile out_labels from get_deduped_metrics_names() : ')
        print(' ---------------------------------------------------------- ')
        for i in out_labels:
            print('     -',i)
        print()
        print(' Post-compile Callback metrics monitored by progbar :')
        print(' ----------------------------------------------------')
        for i in callback_metrics:
            print('     -',i)

        print()
        print(' Post-compile Keras metric_names :') 
        print(' ---------------------------------') 
        for idx, i in enumerate(self.keras_model.metrics_names):
            print('     ',idx, '  ', i)
            
        print()
        print(' Post-compile Keras stateful_metric_names :') 
        print(' ------------------------------------------') 
        for idx, i in enumerate(self.keras_model.stateful_metric_names):
            print('     ',idx, '  ', i)

        ## Setup for stateful_metric_indices Validation process 
        ##--------------------------------------------------------------------------------
        stateful_metric_indices = []
        if hasattr(self, 'metrics'):
            for m in self.stateful_metric_functions:
                m.reset_states()
            stateful_metric_indices = [
                i for i, name in enumerate(self.metrics_names)
                if str(name) in self.stateful_metric_names]
        else:
            stateful_metric_indices = []
            
        ##--------------------------------------------------------------------------------
        ## Callbacks
        ##--------------------------------------------------------------------------------
        # call back for model checkpoint was originally (?) loss. chanegd to val_loss (which is default) 2-5-18
        # copied from \keras\engine\training.py
        # def _get_deduped_metrics_names(self):

            
        callbacks_list = [
              keras.callbacks.ProgbarLogger(count_mode='steps',
                                            stateful_metrics=self.keras_model.stateful_metric_names)
            
            , keras.callbacks.BaseLogger(stateful_metrics=self.keras_model.stateful_metric_names)
            
            , keras.callbacks.TensorBoard(log_dir=self.log_dir,
                                          histogram_freq=1,
                                          write_graph=True,
                                          write_images=False, 
                                          write_grads=True,
                                          batch_size=self.config.BATCH_SIZE)
                                          # write_graph=True,

                                          # write_images=True,
                                          # embeddings_freq=0,
                                          # embeddings_layer_names=None,
                                          # embeddings_metadata=None)

            , keras.callbacks.ModelCheckpoint(self.checkpoint_path, 
                                              mode    = 'auto', 
                                              period  = self.config.CHECKPOINT_PERIOD, 
                                              monitor = 'val_loss', 
                                              verbose = 1, 
                                              save_best_only = True, 
                                              save_weights_only=True)
                                            
            , keras.callbacks.ReduceLROnPlateau(monitor='val_loss', 
                                                mode     = 'auto', 
                                                factor   = self.config.REDUCE_LR_FACTOR,   
                                                cooldown = self.config.REDUCE_LR_COOLDOWN,
                                                patience = self.config.REDUCE_LR_PATIENCE,
                                                min_delta= self.config.REDUCE_LR_MIN_DELTA,
                                                min_lr   = self.config.MIN_LR, 
                                                verbose  = 1)                                            
                                                
            , keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                mode      = 'auto', 
                                                min_delta = self.config.EARLY_STOP_MIN_DELTA, 

                                                patience  = self.config.EARLY_STOP_PATIENCE, 
                                                verbose   = 1)                                            
            , keras.callbacks.History() 
        ]
         
        
        callbacks =  keras.callbacks.CallbackList(callbacks = callbacks_list)
        callbacks.set_model(self.keras_model)
        callbacks.set_params({
            'batch_size': batch_size,
            'epochs': epochs,
            'steps': steps_per_epoch,
            'verbose': 1 ,
            'do_validation': True,
            'metrics': callback_metrics
        })
        
            # 'samples': num_train_samples,
            # 'verbose': verbose,
            # 'do_validation': do_validation,
            # 'metrics': callback_metrics or [],
        
        log(" ")
        log("Training Start Parameters:")
        log("--------------------------")
        log("Starting at epoch     {} of {} epochs.".format(self.epoch, epochs))
        log("Steps per epochs      {} ".format(steps_per_epoch))
        log("Last epoch completed  {} ".format(self.epoch))
        log("Batch size            {} ".format(batch_size))
        log("Learning Rate         {} ".format(self.config.LEARNING_RATE))
        log("Momentum              {} ".format(self.config.LEARNING_MOMENTUM))
        log("Weight Decay:         {} ".format(self.config.WEIGHT_DECAY       ))
        log("VALIDATION_STEPS      {} ".format(self.config.VALIDATION_STEPS   ))
        log("REDUCE_LR_FACTOR      {} ".format(self.config.REDUCE_LR_FACTOR   ))
        log("REDUCE_LR_COOLDOWN    {} ".format(self.config.REDUCE_LR_COOLDOWN ))
        log("REDUCE_LR_PATIENCE    {} ".format(self.config.REDUCE_LR_PATIENCE ))
        log("MIN_LR                {} ".format(self.config.MIN_LR             ))
        log("EARLY_STOP_PATIENCE   {} ".format(self.config.EARLY_STOP_PATIENCE))        
        log("Checkpoint Path:      {} ".format(self.checkpoint_path))


        ##----------------------------------------------------------------------------------------------
        ## If in debug mode write stdout intercepted IO to output file  
        ##----------------------------------------------------------------------------------------------            
        if self.config.SYSOUT == 'FILE':
            utils.write_sysout(self.log_dir)

        ##--------------------------------------------------------------------------------
        ## Start main training loop
        ##--------------------------------------------------------------------------------
        early_stopping  = False
        val_steps = self.config.VALIDATION_STEPS
        epoch_idx = self.epoch
        callbacks.on_train_begin()


        if epoch_idx >= epochs:
            print('Final epoch {} has already completed - Training will not proceed'.format(epochs))
        else:
        
            while epoch_idx < epochs :
                callbacks.on_epoch_begin(epoch_idx)
                epoch_logs = {}
                
                ##------------------------------------------------------------------------
                ## TRAINING Phase - emulating fit_generator()
                ##------------------------------------------------------------------------
                for steps_index in range(steps_per_epoch):
                    
                    # print(' self.epoch {}   epochs {}  step {} '.format(self.epoch, epochs, steps_index))
                    batch_logs = {}
                    batch_logs['batch'] = steps_index
                    batch_logs['size']  = batch_size    

                    callbacks.on_batch_begin(steps_index, batch_logs)

                    train_batch_x, train_batch_y = next(train_generator)

                    
                    # print('len of train batch x' ,len(train_batch_x))
                    # for idx, i in  enumerate(train_batch_x):
                        # print(idx, 'type: ', type(i), 'shape: ', i.shape)
                    # print('len of train batch y' ,len(train_batch_y))
                    # for idx, i in  enumerate(train_batch_y):
                        # print(idx, 'type: ', type(i), 'shape: ', i.shape)
                    # print(type(output_rois))
                    # for i in model_output:
                        # print( i.shape)       
                        
                    ## Run prediction on MRCNN  
                    try:
                        results = mrcnn_model.keras_model.predict(train_batch_x)
                        fcn_x = [train_batch_x[1]]
                        fcn_x.extend(results[:4])
                    
                    except Exception as e :
                        print('failure on mrcnn predict - epoch {} , image ids: {} '.format(epoch_idx, train_batch_x[1][:,0]))
                        print('Exception information:')
                        print(str(e))
                    
                    # print('size of results : ', len(results))
                    # for idx, i in  enumerate(x):
                        # print(idx, 'type: ', type(i), 'shape: ', i.shape)
                    
                    ## Train on FCN
                    try:
                        outs = self.keras_model.train_on_batch(fcn_x , train_batch_y)                                            
                    except Exception as e :
                        print('failure on fcn train - epoch {} , image ids: {} '.format(epoch_idx, train_batch_x[1][:,0]))
                        print('Exception information:')
                        print(str(e))                
                        
                    # print('size of outputs from train_on_batch : ', len(outs), outs)
                    # for idx, i in  enumerate(outs):
                        # print(idx, 'type: ', type(i), 'shape: ', i.shape)
                        
                    if not isinstance(outs, list):
                        outs = [outs]

                    for l, o in zip(out_labels, outs):
                        # print(' out label: ', l, ' out value: ', o,' shape: ', o.shape)
                        batch_logs[l] = o
    
                    callbacks.on_batch_end(steps_index, batch_logs)

                ##------------------------------------------------------------------------
                ## VALIDATION Phase - emulating evaluate_generator()
                ##------------------------------------------------------------------------
                # print(' Start validation ')
                # print(' ---------------- ')
                # print(' Stateful metric indices:' )
                # pp.pprint(stateful_metric_indices)
                
                
                val_steps_done      = 0
                val_outs_per_batch  = []
                val_batch_sizes     = []
                
                # setup validation progress bar if we wish
                # progbar = Progbar(target=val_steps)

                while val_steps_done < val_steps:
                    # print(' ** Validation step: ', val_steps_done)
                    
                    mrcnn_val_x, mrcnn_val_y = next(val_generator)
                    
                    # print('len of train batch x' ,len(val_x))
                    # for idx, i in  enumerate(val_x):
                        # print(idx, 'type: ', type(i), 'shape: ', i.shape)
                        
                    ## Run prediction on MRCNN  
                    try:
                        val_results = mrcnn_model.keras_model.predict(mrcnn_val_x)
                        fcn_val_x = [mrcnn_val_x[1]]
                        fcn_val_x.extend(val_results[:4])   ## image_meta, pr_hm, pr_hm_scores, gt_hm, gt_hm_scores
                    except Exception as e :
                        print('failure on mrcnn predict (validation)- epoch {} , image ids: {} '.format(epoch_idx, mrcnn_val_x[1][:,0]))
                        print('Exception information:')
                        print(str(e))                

                    # print('    mrcnn_model.predict() size of results : ', len(val_results))
                    # for idx, i in  enumerate(xval_results):
                        # print('    ',idx, 'type: ', type(i), 'shape: ', i.shape)
                                       
                    ## Train on FCN
                    try:
                        outs2 = self.keras_model.test_on_batch( fcn_val_x , mrcnn_val_y)
                        # print('\n valstep {} outs2 len:{} '.format(val_steps_done, len(outs2)))
                        val_outs_per_batch.append(outs2)
                    except Exception as e :
                        print('failure on fcn train (validation)- epoch {} , image ids: {} '.format(epoch_idx, mrcnn_val_x[1][:,0]))                    
                        print('Exception information:')
                        print(str(e))                

                    # print('fcn_model.test_on_batch() size of results : ', len(outs2))
                    # for idx, i in  enumerate(outs2):
                        # print(idx, 'type: ', type(i), 'shape: ', i.shape)
                    
                    if isinstance(fcn_val_x, list):
                        batch_size = fcn_val_x[0].shape[0]
                    elif isinstance(fcn_val_x, dict):
                        batch_size = list(fcn_val_x.values())[0].shape[0]
                    else:
                        batch_size = fcn_val_x.shape[0]
                        
                    if batch_size == 0:
                        raise ValueError('Received an empty batch. '
                                         'Batches should at least contain one item.')
                    # else:
                        # print('batch size:', batch_size)
                        
                    val_steps_done += 1
                    val_batch_sizes.append(batch_size)
                    # print validation progress bar if we wish
                    # progbar.update(val_steps_done)

                ## calculate val_averages after all validations steps complete, which is passed 
                ## back to fit_generator() as val_outs 
                
                print('    val_batch_sizes            :', type(val_batch_sizes),' len :', len(val_batch_sizes), val_batch_sizes)
                print('    val_batch_sizes-shape      :', np.asarray(val_batch_sizes).shape)
                
                print('    val_outs_per_batch:        :', type(val_batch_sizes),' len :', len(val_outs_per_batch))
                print('    val_outs_per_batch - shape :', np.asarray(val_outs_per_batch).shape)
                for i,j in enumerate(val_outs_per_batch):
                    print('        batch: ', i, '  ', j)
                
                val_averages = []
                for i in range(len(outs2)):
                    if i not in stateful_metric_indices:
                        tt = [out[i] for out in val_outs_per_batch]
                        # print(' tt type: ',type(tt), tt)
                        # print('val_batch_sizes.shape' , type(val_batch_sizes), len(val_batch_sizes))
                        val_averages.append(
                                np.average([out[i] for out in val_outs_per_batch], axis = 0, weights=val_batch_sizes)
                                           )
                    else:
                        val_averages.append(float(val_outs_per_batch[-1][i]))
                if len(val_averages) == 1:
                    val_averages = val_averages[0]
                print()
                print('val_averages :', val_averages)
                print()
                #-- (unsuccessful) attempt to add histogram info to tensoflow summary  ------------------------
                print(' Tensordlow histogram attempt')
                print('-----------------------------')
                fcn_val_y = self.keras_model.targets
                val_sample_weight = self.keras_model.sample_weights
                print(' len(fcn_val_x)  : ',len(fcn_val_x))
                print(' len(fcn_val_y)  : ',len(fcn_val_y))
                print(' len(mrcnn_val_y): ',len(mrcnn_val_y))

                fcn_val_x, fcn_val_y, fcn_val_sample_weights = my_standardize_user_data(self.keras_model, fcn_val_x, fcn_val_y, val_sample_weight)
                fcn_val_data = fcn_val_x + fcn_val_y  + fcn_val_sample_weights

                print(' len(fcn_val_x)             : ',len(fcn_val_x))
                print(' len(fcn_val_y)             : ',len(fcn_val_y))
                print(' len(fcn_val_sample_weights): ',len(fcn_val_sample_weights))
                print(' len(fcn_val_data)          : ',len(fcn_val_data))
                if self.keras_model.uses_learning_phase and not isinstance(KB.learning_phase(), int):
                    fcn_val_data += [0.]
                for cbk in callbacks:
                    cbk.validation_data = fcn_val_data

                #-------------------------------------------------------------------------------
                
                ##------------------------------------------------------------------------
                ## END OF EPOCH Phase 
                ##------------------------------------------------------------------------
                ## end of evaluate_generator() emulation
                ## val_averages returned back to fit_generator() as val_outs
                ## calculate val_outs after all validations steps complete
                ##------------------------------------------------------------------------
                if not isinstance(val_averages, list):
                    val_averages = [val_averages]
                # Same labels assumed.
                for l, o in zip(out_labels, val_averages):
                    epoch_logs['val_' + l] = o
                    
                #----commented 31-10-18 replaced with above lines -------------------------------------------
                # if not isinstance(outs2, list):
                    # val_outs =  np.average(np.asarray(val_all_outs), weights=val_batch_sizes)
                # else:
                    # averages = []
                    # for i in range(len(outs2)):
                        # averages.append(np.average([out[i] for out in val_all_outs], axis = 0, weights=val_batch_sizes))
                    # val_outs = averages
                # if not isinstance(val_outs, list):
                    # val_outs = [val_outs]
                
                # # Same labels assumed.
                # for l, o in zip(out_labels, val_outs):
                    # # print(' Validations : out label: val_', l, ' out value: ', o)
                    # epoch_logs['val_' + l] = o
                #-------------------------------------------------------------------------------------
                
                # write_log(callback, val_names, logs, batch_no//10)
                # print('\n    validation logs output: ', val_outs)
                
                    
                epoch_logs.update({'lr': KB.eval(self.keras_model.optimizer.lr)})    
                callbacks.on_epoch_end(epoch_idx, epoch_logs)
                epoch_idx += 1
                

                for callback in callbacks:
                    # print(callback)
                    # pp.pprint(dir(callback.model))
                    if hasattr(callback.model, 'stop_training') and (callback.model.stop_training ==True):
                        print(' +++++++++++ ON EPOCH END CALLBACKS TRIGGERED STOP_TRAINING +++++++++++++')
                        print(callback.model, ' triggered stop_training +++++++++++++')
                        early_stopping = True
                        
                if early_stopping:
                    print('{}  Early Stopping triggered on epoch {} of {} epochs'.format(callback, epoch_idx, epochs))
                    break    
                
            ##-------------------------------
            ## end of training operations
            ##--------------------------------
            # if epoch_idx != self.epoch:
            # chkpoint.on_epoch_end(epoch_idx -1, batch_logs)
            callbacks.on_train_end()
            self.epoch = max(epoch_idx - 1, epochs)
            print('Final : self.epoch {}   epochs {}'.format(self.epoch, epochs))
            
        ##--------------------------------------------------------------------------------
        ## End main training loop
        ##--------------------------------------------------------------------------------
        return 
        

## Run data through MRCNN and FCN models

####  Display model input / output information

In [ ]:
mrcnn_model.layer_info()
print('\n FCN')
fcn_model.layer_info()

In [ ]:
# model_output = get_layer_output_2(model.keras_model, train_batch_x, 1)
# model_output = get_layer_output_1(mrcnn_model.keras_model, train_batch_x, [4,5,6,7,9,10,11,12,13,14], 1)
# model_output = get_layer_output_1(mrcnn_model.keras_model, train_batch_x, [0,1,2,3,4,5,6,7,9,10,11], 1)
model_output = get_layer_output_1(mrcnn_model.keras_model, train_batch_x, [0,1,2,3,4,5], 1)


### Load input and output tensors

In [ ]:
print(len(model_output))

# output_rois               = model_output[0]          # layer:  4   shape: (1, 200, 4)
# target_class_ids          = model_output[1]          # layer:  5   shape: (1, 200)
# target_bbox_deltas        = model_output[2]          # layer:  6   shape: (1, 200, 4)
# roi_gt_boxes              = model_output[3]          # layer:  7   shape: (1, 200, 4)
# mrcnn_class               = model_output[4]          # layer:  8   shape: (1, 200, 81)
# mrcnn_bbox                = model_output[5]          # layer:  9   shape: (1, 200, 81, 4)
# pred_refined_tensor       = model_output[6]          # layer: 16   shape: (1, 81, 25, 7)
# output_rois               = model_output[0]          # layer:  0   shape: (2, 200, 4)
# target_class_ids          = model_output[1]          # layer:  1   shape: (2, 200)
# target_bbox_deltas        = model_output[2]          # layer:  2   shape: (2, 200, 4)
# roi_gt_boxes              = model_output[3]          # layer:  3   shape: (2, 200, 4)
# mrcnn_class               = model_output[4]          # layer:  4   shape: (2, 200, 81)
# mrcnn_bbox                = model_output[5]          # layer:  5   shape: (2, 200, 81, 4)
# model_pred_heatmap_norm         = model_output[6]          # layer:  6   shape: (2, 256, 256, 81)
# model_pred_heatmap_scores       = model_output[7]          # layer:  7   shape: (2, 81, 25, 11)
# model_gt_heatmap_scores         = model_output[8]          # layer:  9   shape: (2, 81, 25, 11)
# model_pred_tensor               = model_output[9]          # layer: 10   shape: (2, 81, 25, 8)
# model_gt_tensor                 = model_output[10]          # layer: 11   shape: (2, 81, 25, 8)

pred_heatmap_norm         = model_output[0]          # layer:  0   shape: (2, 256, 256, 81)
pred_heatmap_scores       = model_output[1]          # layer:  1   shape: (2, 81, 200, 11)
gt_heatmap_norm           = model_output[2]          # layer:  2   shape: (2, 256, 256, 81)
gt_heatmap_scores         = model_output[3]          # layer:  3   shape: (2, 81, 200, 11)
pred_tensor               = model_output[4]          # layer:  4   shape: (2, 81, 200, 8)
gt_tensor                 = model_output[5]          # layer:  5   shape: (2, 81, 200, 8)
for i in model_output:
    print( i.shape)

In [ ]:
# fcn_input = [pred_heatmap_norm, pred_heatmap_scores, gt_heatmap_norm, gt_heatmap_scores] 
model_output2 = get_layer_output_1(fcn_model.keras_model, fcn_input, [0,1], 1)

### Load input data

In [ ]:
input_image          =  train_batch_x[0]
input_image_meta     =  train_batch_x[1]
# input_rpn_match      =  train_batch_x[2]
# input_rpn_bbox       =  train_batch_x[3]
input_gt_class_ids   =  train_batch_x[4]
input_gt_bboxes      =  train_batch_x[5]
print(' Input image shape is    :', input_image.shape)
print(' input_image_meta        :', input_image_meta[0,:10])
# print(' input_rpn_match         :', input_rpn_match.shape)
# print(' input_rpn_bbox          :', input_rpn_bbox.shape)
print(' input_gt_class_ids      :', input_gt_class_ids.shape)
print(' input_gt_bboxes         :', input_gt_bboxes.shape)
# h, w = input_image.shape[1], input_image.shape[2]      #  tf.shape(input_image)[1], tf.shape(input_image)[2]
# input_gt_bboxes_norm = tf.identity(input_gt_bboxes / [h,w,h,w])
# print(' input_gt_bboxes_norm    :', input_gt_bboxes_norm.shape)

## Display output from model

####  `input_gt_class_ids`, `input_gt_bboxes`

In [ ]:
# print(roi_gt_boxes[0,:50] * [1024,1024,1024,1024])
print(input_gt_class_ids[0])
print(input_gt_bboxes[0,:10])
# for i in range(input_gt_class_ids.shape[1]):
#     if input_gt_class_ids[0,i] == 1:
#         print(input_gt_class_ids[0,i], '   ', input_gt_bboxes[0,i])

####  Display `output_rois`

In [ ]:
np.set_printoptions(precision=4, threshold=4000, linewidth=210, suppress=True)
img = 0
print(' output_rois')
print(output_rois.shape)
# print(output_rois[0,:40,:])
print(output_rois [0,:40,:]* [1024, 1024, 1024, 1024])

####  `max_mrcnn_class` , `argmax_mrcnn_class`

In [ ]:
np.set_printoptions(precision=4, threshold=4000, linewidth=210, suppress=True)
print(' mrcnn_class', mrcnn_class.shape)
# print( mrcnn_class[0,0,:])
# 
max_mrcnn_class    = np.max(mrcnn_class, axis = (0,2))
argmax_mrcnn_class = np.argmax(mrcnn_class, axis = 2)

# print()
print('\n mrcnn_class Max Values   : ', max_mrcnn_class.shape)
print(max_mrcnn_class)

# print()
print(' mrcnn_class Argmax Values: ', argmax_mrcnn_class.shape)
print(argmax_mrcnn_class[0])

print(' target_class_ds    Values: ', target_class_ids.shape)
print(target_class_ids[0])

# for i in range(100):
#     print('Predicted: ', argmax_mrcnn_class[0,i],  '  Actual ', target_class_ids[0,i])
 

#### Display  `target_class_ids()`

In [ ]:
print(' target_class_ids')
print(target_class_ids.shape)
print(target_class_ids[0,:70])

####  apply `deltas` from predicted delta `mrcnn_bbox`  to  `output_rois` to obtain refined rois 

In [ ]:
np.set_printoptions(precision=4, threshold=4000, linewidth=210, suppress=True)
img_idx = 0 

print('output_rois',output_rois.shape, 'deltas ', deltas.shape)
cls = 1
for i in range(input_gt_class_ids.shape[1]):
    if input_gt_class_ids[0,i] == cls:
        print(input_gt_class_ids[0,i], '   ', input_gt_bboxes[0,i])

        
print()        
for i in range(output_rois.shape[1]):
    if classes[0,i] ==cls:
        print(' i ', i, 'class: ',classes[0,i])
#         print('   orig           : ', output_rois[0,i])
        d1 = deltas[0,i] * mrcnn_config.BBOX_STD_DEV
#         print('   delta          : ', deltas[0,i],'   delta * std dev: ', d1)
        d2 = utils.apply_box_delta(output_rois[0,i],d1)
#         print('   refined        : ', d2)
#         print()
        print('   orig           : ',output_rois[0,i] * [1024,1024,1024,1024])
        print('   refined        : ', d2 * [1024,1024,1024,1024]) 
        print('   roi_gt_bboxes  : ', roi_gt_boxes[0,i]* [1024,1024,1024,1024]) 
        print()
        print('   pred delta     : ', deltas[0,i] )
        print('   tgt delta      : ', target_bbox_deltas[0,i] )
        
        print()


####  Display roi_gt_boxes , and class_ids vs. output_bbox and prediceted class_ids

In [ ]:
print(ref_out_roi)

In [ ]:
ref_out_roi1 = ref_out_roi * [1024,1024,1024,1024]
print(ref_out_roi1)
window = np.array([0,0,1024,1024], dtype =float)
print(window.shape)
ref_out_roi2  = utils.clip_to_window_np( window, ref_out_roi1)
print(ref_out_roi2.shape)
for i in range(200):
    print(ref_out_roi1[i],' --- ', ref_out_roi2[i])

#### Display pred_refined_tensor and gt_tensor

In [ ]:
for cls in [1]:
    for box in range(20):
        print(pred_tensor[0,cls,box])
        print(gt_tensor[0,cls,box])
        print()

####  Display roi_gt_boxes along with corresponding refined/clipped output_rois

In [ ]:
img_id = 0
print(roi_gt_boxes[0].shape, target_class_ids[0].shape , np.expand_dims(target_class_ids[0],axis=-1).shape)
classes, deltas = utils.get_predicted_mrcnn_deltas(mrcnn_class, mrcnn_bbox, verbose=True)
deltas *= mrcnn_config.BBOX_STD_DEV
print('classes.shape: ',classes.shape, ' deltas.shape: ',deltas.shape)

ref_out_roi = utils.apply_box_deltas_np(output_rois[img_id],deltas[img_id])
#     ##   Clip boxes to image window    
# print(ref_out_roi.shape)
window = np.array([0,0,1024,1024], dtype =float)
clipped_out_roi  = utils.clip_to_window_np( window, ref_out_roi*[1024,1024,1024,1024])

for i in range(200):
#     ref_out_roi = utils.apply_box_delta_np(output_rois[0],d1[0])
#     if classes[img_id,i] == 1 or target_class_ids[img_id,i] == 1 :

    print('idx: ',200-i,' GT Cls: ', target_class_ids[img_id,i]  , ' -', roi_gt_boxes[img_id,i]*[1024,1024,1024,1024], 
                    ' PR Cls: ', classes[img_id,i],' - ', ref_out_roi[i]*[1024.0,1024.0,1024.0,1024.0] ,
                     'ClpdCls: ', clipped_out_roi[i]   ) #) *[1024,1024,1024,1024]

#### display gt_heatmap_scores and pred_heatmap_scores outputs

In [ ]:
np.set_printoptions(precision=4, threshold=None, linewidth=200, suppress=True)
# print(' gt_tensor')
# print(gt_tensor.shape)
# print(gt_tensor[img,:,:10])
img_id = 1
print(' GT Heatmap Scores')

print('gt_heatmap_scores: ', gt_heatmap_scores.dtype,  gt_heatmap_scores.shape)
print('pred_heatmap_scores: ', pred_heatmap_scores.dtype,  pred_heatmap_scores.shape)

# print(gt_heatmap_scores[img,1])
# for img_id in range(mrcnn_config.BATCH_SIZE):

for img_id in [0]:    # print(pred_refined_heatmap_scores[img_id,:4])
    pr_class_ids = np.unique(pred_heatmap_scores[img_id,:,:,4]).astype(int).tolist()
    gt_class_ids = np.unique(gt_heatmap_scores[img_id,:,:,4]).astype(int).tolist() 
    union_class_ids = np.union1d(pr_class_ids, gt_class_ids)
    print('-'*56)
    print('Image : {}  GT ClassIds: {}   PR ClassIds: {} '.format(img_id, gt_class_ids, pr_class_ids))
    print('Image : {}  Union ClassIds: {}'.format(img_id, union_class_ids))
    print('-'*56)
    for cls in union_class_ids:  
        print()
        for i in range(25):
#             print(' GT: img_id:',img_id, ' cls: ',cls, ' -',gt_tensor[img_id, cls,i]) #, gt_heatmap_scores[img_id, cls,i,7] )
#             print(' PR: img_id:',img_id, ' cls: ',cls, ' -',pred_tensor[img_id,cls,i]) #,pred_refined_heatmap_scores[img_id,cls,i,7])

            print(' GT: img/cls:',img_id, '/',cls, ' -',gt_heatmap_scores[img_id, cls,i]) #, gt_heatmap_scores[img_id, cls,i,7] )
            print(' PR: img/cls:',img_id, '/',cls, ' -',pred_heatmap_scores[img_id,cls,i]) #,pred_refined_heatmap_scores[img_id,cls,i,7])
            print()

####  Display `Pred_Tensor`, `Pred_heatmap`, `mrcnn_class`

In [ ]:
# np.set_printoptions(precision=4, threshold=None, linewidth=150, suppress=True)
np.set_printoptions(precision=4, threshold=4000, linewidth=210, suppress=True)
img = 0

# max_score = np.max(mrcnn_class, axis = -1)
# max_class = np.argmax(mrcnn_class, axis = -1)
# # print(' output_rois[',img,'] \n', output_rois[1]*[128,128,128,128])
# print('max class shape:',max_class.shape, 'max score shape: ',max_score.shape)
# print('max class[',img,']\n',max_class[img])
# print('max score[',img,']\n',max_score[img])
# print('mrcnn class.shape ',mrcnn_class.shape)
# print('mrcnn_class[',img,',:]\n',mrcnn_class[img,:])
# print(output_rois[1])

print('input_gt_class_ids')
print(input_gt_class_ids[0])

# print(' rpn_bbox')
# print(rpn_bbox.shape)
# print(rpn_bbox[0,:100,:])

# print(' rpn_roi_proposals')
# print(rpn_roi_proposals.shape)
# print(rpn_roi_proposals[0,:100,:])

print(' output_rois')
print(output_rois.shape)
# print(output_rois[0,:40,:])
print(output_rois [0,:40,:]* [1024, 1024, 1024, 1024])

print(' target_class_ids')
print(target_class_ids.shape)
print(target_class_ids[0,:40])
# print(output_rois [0,:40,:]* [1024, 1024, 1024, 1024])

# print(' Pred_tensor')
# print(pred_tensor.shape)
# print(pred_tensor[img,:,:10])

# print(' gt_tensor')
# print(gt_tensor.shape)
# print(gt_tensor[img,:,:10])

# print(' mrcnn_class')
# print( mrcnn_class.shape)
# print( mrcnn_class[0,:,:])

# print(' mrcnn_bbox')
# print( mrcnn_bbox.shape)
# print( mrcnn_bbox)

# print(' roi_gt_boxes')
# print(roi_gt_boxes.shape)
# print(roi_gt_boxes[img,:,:])

# print(' Pred Heatmap Scores')
# print(pred_heatmap_scores.dtype, pred_heatmap_scores.shape)
# print(pred_heatmap_scores[img,1])

# print(' FCN Scores')
# print(fcn_scores.dtype)
# for cls in range(4):
#     print(pred_heatmap_scores[img,cls,:10])
#     print(fcn_scores[img,cls,:10,2:])

  

####  Display `output_rois` for visual check - passed on to  `build_pred_tensor()`

In [ ]:
np.set_printoptions(linewidth=150, precision=6)
# print('scatter shape is ', pred_scatt.get_shape())
print('output_rois shape is ', output_rois.shape)
img = 0
for img in [0]:
    print('Image ', img , ' ------------')
    print(output_rois[img])

####  Display  - `pred_refined_tensor` which is passed on to  `build_heatmap()`

In [ ]:
np.set_printoptions(linewidth=150, precision=6)
img_id = 0
# print('scatter shape is ', pred_scatt.get_shape())
print('model_pred_tensor shape is ', model_pred_tensor.shape)
print(input_image_meta[0,:10])
pr_class_ids = np.unique(model_pred_heatmap_scores[img_id,:,:,4]).astype(int).tolist()
print('Image : {}  PR ClassIds: {} '.format(img_id, pr_class_ids))
for k in pr_class_ids:
    print('Image ', img , '/ Class ',k,' ------------')
    print(model_pred_tensor[img,k,:30])
    

####  Compare  `pred_heatmap_scores` vs. `pred_refined_heatmap_scores`

###  Setup tensors to be passed to `build_predictions ()`    

In [ ]:
mrcnn_bbox  = tf.identity(mrcnn_bbox)
mrcnn_class = tf.identity(mrcnn_class)
norm_input_rois = tf.identity(output_rois)
config      = mrcnn_config
sess = KB.get_session()
print(' Keras session :', sess)
import mrcnn.utils as utils

###  Run TF graph

In [ ]:
# with sess1.as_default():
# FeedList = [positive_ind_shuffled, positive_indices, positive_overlaps, roi_gt_class_ids, roi_gt_boxes, roi_gt_box_assignment ]
# FeedList = [ rois, roi_gt_class_ids,  roi_gt_deltas, roi_gt_boxes]
Fetches  = [ pred_heatmap, pred_heatmap_norm, pred_heatmap_scores]
tt = sess.run(Fetches)
print(type(tt), len(tt))
for i in tt:
    print(type(i), i.shape)